# Initialize

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cuda --name my_curand.cu 
/*
 * This program uses the host CURAND API to generate 100 
 * pseudorandom floats.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

#define CUDA_CALL(x) do { if((x)!=cudaSuccess) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)
#define CURAND_CALL(x) do { if((x)!=CURAND_STATUS_SUCCESS) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)

int main(int argc, char *argv[])
{
    size_t n = 100;
    size_t i;
    curandGenerator_t gen;
    float *devData, *hostData;

    /* Allocate n floats on host */
    hostData = (float *)calloc(n, sizeof(float));

    /* Allocate n floats on device */
    CUDA_CALL(cudaMalloc((void **)&devData, n*sizeof(float)));

    /* Create pseudo-random number generator */
    CURAND_CALL(curandCreateGenerator(&gen, 
                CURAND_RNG_PSEUDO_DEFAULT));

    /* Set seed */
    CURAND_CALL(curandSetPseudoRandomGeneratorSeed(gen, 
                1234ULL));

    /* Generate n floats on device */
    CURAND_CALL(curandGenerateUniform(gen, devData, n));

    /* Copy device memory to host */
    CUDA_CALL(cudaMemcpy(hostData, devData, n * sizeof(float),
        cudaMemcpyDeviceToHost));

    /* Show result */
    for(i = 0; i < n; i++) {
        printf("%1.4f ", hostData[i]);
    }
    printf("\n");

    /* Cleanup */
    CURAND_CALL(curandDestroyGenerator(gen));
    CUDA_CALL(cudaFree(devData));
    free(hostData);    
    return EXIT_SUCCESS;
}

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand

In [ ]:
!/content/src/my_curand

# GPU hello world

In [ ]:
%%cuda --name ex1.cu
/*
 * This program executs a kernel.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

__global__  void my_kernel(void){
}

int main(void){
    my_kernel<<<1,1>>>();

    printf("hello world\n");
    return 0;
}

In [ ]:
!nvcc -o /content/src/ex1.cu /content/src/ex1.cu -lcurand

In [ ]:
!/content/src/ex1.cu

# Ex2 - Add two numbers

In [ ]:
%%cuda --name ex2.cu
/*
 * This program executs a kernel.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

__global__  void my_add(int *a, int *b, int *c){
    *c = *a + *b;
}

int main(void){
    int a, b, c;
    int *d_a, *d_b, *d_c;
    int size = sizeof(int);

    // Allocatee space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    a = 2;
    b = 7;

    cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

    // Launch add on GPU
    my_add<<<1,1>>>(d_a, d_b, d_c);

    // Copy result back to host
    cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
    printf("c=%d\n",c);

    // Cleanup
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    return 0;
}

In [ ]:
!nvcc -o /content/src/ex2.cu /content/src/ex2.cu -lcurand

In [ ]:
!/content/src/ex2.cu

# Ex3 - Add with blocks

In [ ]:
%%cuda --name ex3.cu
/*
 * This program executs a kernel.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

__global__  void my_add(int *a, int *b, int *c){
    c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}

void random_ints(int* x, int size)
{
	int i;
	for (i=0;i<size;i++) {
		x[i]=rand()%10;
	}
}

# define N 512
int main(void){
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocatee space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    a = (int *)malloc(size); random_ints(a, N);
    b = (int *)malloc(size); random_ints(b, N);
    c = (int *)malloc(size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch add on GPU
    my_add<<<N,1>>>(d_a, d_b, d_c);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
//    for(int i=0;i<N;i++)
//      printf("%d) c=%d\n",i,c[i]);

    // Cleanup
    free(a); free(b); free(c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    return 0;
}

In [ ]:
!nvcc -o /content/src/ex3.cu /content/src/ex3.cu -lcurand

In [ ]:
!/content/src/ex3.cu

# Ex4 - Add with threads

In [ ]:
%%cuda --name ex4.cu
/*
 * This program executs a kernel.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

__global__  void my_add(int *a, int *b, int *c){
    c[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x];
}

void random_ints(int* x, int size)
{
	int i;
	for (i=0;i<size;i++) {
		x[i]=rand()%10;
	}
}

# define N 512
int main(void){
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocatee space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    a = (int *)malloc(size); random_ints(a, N);
    b = (int *)malloc(size); random_ints(b, N);
    c = (int *)malloc(size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch add on GPU
    my_add<<<1,N>>>(d_a, d_b, d_c);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
//    for(int i=0;i<N;i++)
//      printf("%d) c=%d\n",i,c[i]);

    // Cleanup
    free(a); free(b); free(c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    return 0;
}

In [ ]:
!nvcc -o /content/src/ex4.cu /content/src/ex4.cu -lcurand

In [ ]:
!/content/src/ex4.cu

# Ex5 - With blocks and threads

In [ ]:
%%cuda --name ex5.cu
/*
 * This program executs a kernel.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

__global__  void my_add(int *a, int *b, int *c){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    c[index] = a[index] + b[index];
}

void random_ints(int* x, int size)
{
	int i;
	for (i=0;i<size;i++) {
		x[i]=rand()%10;
	}
}

# define N (2048 * 2048)
# define THREADS_PER_BLOCK 512
int main(void){
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocatee space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    a = (int *)malloc(size); random_ints(a, N);
    b = (int *)malloc(size); random_ints(b, N);
    c = (int *)malloc(size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch add on GPU
    my_add<<<N/THREADS_PER_BLOCK,THREADS_PER_BLOCK>>>(d_a, d_b, d_c);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
//    for(int i=0;i<N;i++)
//      printf("%d) c=%d\n",i,c[i]);

    // Cleanup
    free(a); free(b); free(c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    return 0;
}

In [ ]:
!nvcc -o /content/src/ex5.cu /content/src/ex5.cu -lcurand

In [ ]:
!/content/src/ex5.cu

# Ex6 - Mat mul

In [ ]:
%%cuda --name ex6.cu
/*
 * This program multiplies two matrices.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

#define WIDTH 16

typedef struct {
    int width;
    int height;
    float* elements;
} Matrix;

// Matrix multiplication kernel ? thread specification
__global__ void MatrixMulKernel(Matrix M, Matrix N, Matrix P)
{
    // 2D Thread ID
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;
 
    for (int k = 0; k < M.width; ++k)
    { 
         float Melement = M.elements[ty * M.width + k];
         float Nelement = N.elements[k * N.width + tx];
         Pvalue += Melement * Nelement;
    } 
    // Write the matrix to device memory;
    // each thread writes one element
    P.elements[ty * P.width + tx] = Pvalue;
}

// Allocate a device matrix of same size as M.
Matrix AllocateDeviceMatrix(const Matrix M)
{
    Matrix Mdevice = M;
    int size = M.width * M.height * sizeof(float);
    cudaMalloc((void**)&Mdevice.elements, size);
    return Mdevice;
}

// Free a device matrix.
void FreeDeviceMatrix(Matrix M) {
    cudaFree(M.elements);
}

void FreeMatrix(Matrix M) {
    free(M.elements);
}

// Copy a host matrix to a device matrix.
void CopyToDeviceMatrix(Matrix Mdevice, const Matrix Mhost)
{
    int size = Mhost.width * Mhost.height * sizeof(float);
    cudaMemcpy(Mdevice.elements, Mhost.elements, size, 
	cudaMemcpyHostToDevice);
}

// Copy a device matrix to a host matrix.
void CopyFromDeviceMatrix(Matrix Mhost, const Matrix Mdevice)
{
    int size = Mdevice.width * Mdevice.height * sizeof(float);
    cudaMemcpy(Mhost.elements, Mdevice.elements, size, 
	cudaMemcpyDeviceToHost);
}

// Matrix multiplication on the device
void MatrixMulOnDevice(const Matrix M, const Matrix N, Matrix P)
{
    // Load M and N to the device
    Matrix Md = AllocateDeviceMatrix(M);
    CopyToDeviceMatrix(Md, M);
    Matrix Nd = AllocateDeviceMatrix(N);
    CopyToDeviceMatrix(Nd, N);

    // Allocate P on the device
    Matrix Pd = AllocateDeviceMatrix(P);
    CopyToDeviceMatrix(Pd, P); // Clear memory
    
     // Setup the execution configuration
    dim3 dimBlock(WIDTH, WIDTH);
    dim3 dimGrid(1, 1);

    // Launch the device computation threads!
    MatrixMulKernel<<<dimGrid, dimBlock>>>(Md, Nd, Pd);

    // Read P from the device
    CopyFromDeviceMatrix(P, Pd); 

    // Free device matrices
    FreeDeviceMatrix(Md);
    FreeDeviceMatrix(Nd);
    FreeDeviceMatrix(Pd);
} 

Matrix AllocateMatrix(int height, int width)
{
    Matrix M;
    M.width = width;
    M.height = height;
    int size = M.width * M.height;
    M.elements = NULL;

    M.elements = (float*) malloc(size*sizeof(float));

    for(unsigned int i = 0; i < M.height * M.width; i++)
    {
        M.elements[i] = (rand() / (float)RAND_MAX);
        if(rand() % 2)
            M.elements[i] = - M.elements[i];
    }
    return M;
}


void PrintMatrix(float* ma, int X, int Y)
{
	int i,j;
	for (j=0;j<Y;j++) {
		for (i=0;i<X;i++) {
			printf("%4f ",ma[i+j*X]);
		}
		printf("\n");
	}
}


int main(void) 
{
	int i,j;
    // Allocate and initialize the matrices
    Matrix  M  = AllocateMatrix(WIDTH, WIDTH);
    Matrix  N  = AllocateMatrix(WIDTH, WIDTH);
    Matrix  P  = AllocateMatrix(WIDTH, WIDTH);

    // M * N on the device
    MatrixMulOnDevice(M, N, P);


	PrintMatrix(M.elements,M.width,M.height);
	printf("\n");
	PrintMatrix(N.elements,N.width,N.height);
	printf("\n");
	PrintMatrix(P.elements,P.width,P.height);

    // Free matrices
    FreeMatrix(M);
    FreeMatrix(N);
    FreeMatrix(P);

    return 0;
}




In [ ]:
!nvcc -o /content/src/ex6.cu /content/src/ex6.cu -lcurand

In [ ]:
!/content/src/ex6.cu

# Data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
!cd /content/drive

In [ ]:
!pwd

In [ ]:
!ls /content/drive/'My Drive'

# Use header files

In [ ]:
%%cuda --name linear_algebra_helpers.h
/*
 * This program multiplies two matrices.
 */
#include <stdio.h>
#include <stdlib.h>
#include <fstream>
#include <cuda.h>
#include <curand.h>

#define WIDTH 16

/* *****************************
 * Vector Type
 ********************************/
typedef struct {
    int length;
    float* elements;
} Vector;

/* *****************************
 * Vector Functions
 ********************************/
// Vector multiplication kernel ? thread specification
__global__ void VectorMulKernel(Vector M, Vector N, Vector P)
{
    // 1D Thread ID
    int tx = threadIdx.x;

    // Pvalue is used to store the element of the vector
    // that is computed by the thread
    float Pvalue = 0;

    float Melement = M.elements[tx];
    float Nelement = N.elements[tx];
    Pvalue += Melement * Nelement;

    // Write the matrix to device memory;
    // each thread writes one element
    P.elements[tx] = Pvalue;
}

// Allocate a device vector of same size as V.
Vector AllocateDeviceVector(const Vector V)
{
    Vector Vdevice = V;
    int size = V.length * sizeof(float);
    cudaMalloc((void**)&Vdevice.elements, size);
    return Vdevice;
}

// Free a device vector.
void FreeDeviceVector(Vector V) {
    cudaFree(V.elements);
}

void FreeVector(Vector V) {
    free(V.elements);
}

// Copy a host vector to a device vector.
void CopyToDeviceVector(Vector Vdevice, const Vector Vhost)
{
    int size = Vhost.length * sizeof(float);
    cudaMemcpy(Vdevice.elements, Vhost.elements, size, 
	  cudaMemcpyHostToDevice);
}

// Copy a device vector to a host vector.
void CopyFromDeviceVector(Vector Vhost, const Vector Vdevice)
{
    int size = Vdevice.length * sizeof(float);
    cudaMemcpy(Vhost.elements, Vdevice.elements, size, 
	  cudaMemcpyDeviceToHost);
}

// Vector multiplication on the device
void VectorMulOnDevice(const Vector M, const Vector N, Vector P)
{
    // Load M and N to the device
    Vector Md = AllocateDeviceVector(M);
    CopyToDeviceVector(Md, M);
    Vector Nd = AllocateDeviceVector(N);
    CopyToDeviceVector(Nd, N);

    // Allocate P on the device
    Vector Pd = AllocateDeviceVector(P);
    CopyToDeviceVector(Pd, P); // Clear memory
    
     // Setup the execution configuration
    dim3 dimBlock(WIDTH);
    dim3 dimGrid(1);

    // Launch the device computation threads!
    VectorMulKernel<<<dimGrid, dimBlock>>>(Md, Nd, Pd);

    // Read P from the device
    CopyFromDeviceVector(P, Pd); 

    // Free device vectors
    FreeDeviceVector(Md);
    FreeDeviceVector(Nd);
    FreeDeviceVector(Pd);
} 

Vector AllocateVector(int length)
{
    Vector M;
    M.length = length;
    int size = M.length;
    M.elements = NULL;

    M.elements = (float*) malloc(size*sizeof(float));

    for(unsigned int i = 0; i < M.length; i++)
    {
        M.elements[i] = (rand() / (float)RAND_MAX);
        if(rand() % 2)
            M.elements[i] = - M.elements[i];
    }
    return M;
}


void PrintVector(float* ma, int X)
{
	int i;
	for (i=0;i<X;i++) {
			printf("%4f ",ma[i]);
	}
}

// Serialize Vector
void SerializeVector(Vector V, const char *filename)
{
    std::ofstream f(filename);
    for(unsigned int i = 0; i < V.length; i++) {
       f << V.elements[i] << '\n';
    } 
}


/* *****************************
 * Matrix Type
 ********************************/
typedef struct {
    int width;
    int height;
    float* elements;
} Matrix;

/* *****************************
 * Matrix Functions
 ********************************/
// Matrix multiplication kernel ? thread specification
__global__ void MatrixMulKernel(Matrix M, Matrix N, Matrix P)
{
    // 2D Thread ID
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;
 
    for (int k = 0; k < M.width; ++k)
    { 
         float Melement = M.elements[ty * M.width + k];
         float Nelement = N.elements[k * N.width + tx];
         Pvalue += Melement * Nelement;
    } 
    // Write the matrix to device memory;
    // each thread writes one element
    P.elements[ty * P.width + tx] = Pvalue;
}

// Allocate a device matrix of same size as M.
Matrix AllocateDeviceMatrix(const Matrix M)
{
    Matrix Mdevice = M;
    int size = M.width * M.height * sizeof(float);
    cudaMalloc((void**)&Mdevice.elements, size);
    return Mdevice;
}

// Free a device matrix.
void FreeDeviceMatrix(Matrix M) {
    cudaFree(M.elements);
}

void FreeMatrix(Matrix M) {
    free(M.elements);
}

// Copy a host matrix to a device matrix.
void CopyToDeviceMatrix(Matrix Mdevice, const Matrix Mhost)
{
    int size = Mhost.width * Mhost.height * sizeof(float);
    cudaMemcpy(Mdevice.elements, Mhost.elements, size, 
	cudaMemcpyHostToDevice);
}

// Copy a device matrix to a host matrix.
void CopyFromDeviceMatrix(Matrix Mhost, const Matrix Mdevice)
{
    int size = Mdevice.width * Mdevice.height * sizeof(float);
    cudaMemcpy(Mhost.elements, Mdevice.elements, size, 
	cudaMemcpyDeviceToHost);
}

// Matrix multiplication on the device
void MatrixMulOnDevice(const Matrix M, const Matrix N, Matrix P)
{
    // Load M and N to the device
    Matrix Md = AllocateDeviceMatrix(M);
    CopyToDeviceMatrix(Md, M);
    Matrix Nd = AllocateDeviceMatrix(N);
    CopyToDeviceMatrix(Nd, N);

    // Allocate P on the device
    Matrix Pd = AllocateDeviceMatrix(P);
    CopyToDeviceMatrix(Pd, P); // Clear memory
    
     // Setup the execution configuration
    dim3 dimBlock(WIDTH, WIDTH);
    dim3 dimGrid(1, 1);

    // Launch the device computation threads!
    MatrixMulKernel<<<dimGrid, dimBlock>>>(Md, Nd, Pd);

    // Read P from the device
    CopyFromDeviceMatrix(P, Pd); 

    // Free device matrices
    FreeDeviceMatrix(Md);
    FreeDeviceMatrix(Nd);
    FreeDeviceMatrix(Pd);
} 

Matrix AllocateMatrix(int height, int width)
{
    Matrix M;
    M.width = width;
    M.height = height;
    int size = M.width * M.height;
    M.elements = NULL;

    M.elements = (float*) malloc(size*sizeof(float));

    for(unsigned int i = 0; i < M.height * M.width; i++)
    {
        M.elements[i] = (rand() / (float)RAND_MAX);
        if(rand() % 2)
            M.elements[i] = - M.elements[i];
    }
    return M;
}


void PrintMatrix(float* ma, int X, int Y)
{
	int i,j;
	for (j=0;j<Y;j++) {
		for (i=0;i<X;i++) {
			printf("%4f ",ma[i+j*X]);
		}
		printf("\n");
	}
}


// Serialize Matrix
void SerializeMatrix(Matrix M, const char *filename)
{
    std::ofstream f(filename);
    for(unsigned int i = 0; i < M.width; i++) 
    {
        for(unsigned int j = 0; j < M.height; j++)
        {
            f << M.elements[i * M.height + j] << '\n';
        }

    } 
}



## Testing

In [ ]:
%%cuda --name ex7.cu
/*
 * This program multiplies two matrices.
 */

 #include "linear_algebra_helpers.h"

int main(void) 
{
	int i,j;
    // Allocate and initialize the matrices
    Matrix  M  = AllocateMatrix(WIDTH, WIDTH);
    Matrix  N  = AllocateMatrix(WIDTH, WIDTH);
    Matrix  P  = AllocateMatrix(WIDTH, WIDTH);
 
    Vector V1  = AllocateVector(WIDTH);

    // M * N on the device
    MatrixMulOnDevice(M, N, P);


	PrintMatrix(M.elements,M.width,M.height);
	printf("\n");
	PrintMatrix(N.elements,N.width,N.height);
	printf("\n");
	PrintMatrix(P.elements,P.width,P.height);

    // Free matrices
    FreeMatrix(M);
    FreeMatrix(N);
    FreeMatrix(P);
 
    FreeVector(V1);

    return 0;
}

In [ ]:
!nvcc -o /content/src/ex7.cu /content/src/ex7.cu -lcurand

In [ ]:
%%time
!/content/src/ex7.cu

# Calculate Pi

## Initialize a matrix of points

In [ ]:
%%cuda --name calc_pi_MCMC.cu
/*
 * This program multiplies two matrices.
 */
#include "linear_algebra_helpers.h"

// 2^13
#define NMC  8192 

// MCMC Kernel
__global__  void my_add(int *a, int *b, int *c){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    c[index] = a[index] + b[index];
}

int main(void) 
{
	  int i,j;
    // Allocate and initialize the matrices
    Matrix  M  = AllocateMatrix(WIDTH, WIDTH);
 
    // Initialize Matrix of grid points
    for(unsigned int i = 0; i < M.height; i++)
    {
        for(unsigned int j=0;j < M.width; j++)
        {
            M.elements[i * M.width + j] = (WIDTH/2 - i)*(WIDTH/2 - i) + (WIDTH/2 - j)*(WIDTH/2 - j);
        }
    }
 
	  PrintMatrix(M.elements,M.width,M.height);
  	printf("\n");
 
    // X and Y vectors
    Vector IdxI = AllocateVector(WIDTH);
    Vector IdxJ = AllocateVector(WIDTH);
 
    SerializeVector(IdxI, "random_numbers.dat");
    SerializeMatrix(M, "DistanceMatrix.dat");

    // Vector of measurements
    Vector PiValues = AllocateVector(NMC);
    

    // Free matrices
    FreeMatrix(M);
    FreeVector(IdxI);
    FreeVector(IdxJ);
    FreeVector(PiValues);

    return 0;
}

In [ ]:
!nvcc -o /content/src/calc_pi_MCMC /content/src/calc_pi_MCMC.cu -lcurand
!/content/src/calc_pi_MCMC

In [ ]:
import matplotlib.pyplot as plt
import numpy

data = numpy.loadtxt("DistanceMatrix.dat")
size = data.shape
dimM = int(numpy.sqrt(size[0]));
dimN = int(size[0]/dimM)

print(numpy.sqrt(size))
# plt.plot(data)
plt.imshow(data.reshape(dimM,dimN), cmap='hot', interpolation='nearest')
plt.show()

# Working with Drive and Git

## Initalize google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/My Drive"

## Initialize with the git repo on github


In [ ]:
#!git clone https://github.com/v1j4y/calculate_pi_GPU.git

In [ ]:
%cd "calculate_pi_GPU"

In [ ]:
!ls

In [ ]:
!git pull

In [ ]:
!ls src/ lib/

In [ ]:
!pwd

## Modify work

In [ ]:
%%cuda --name calc_pi_MCMC_with_git_files.cu
/*
 * This program multiplies two matrices.
 */
#include "linear_algebra_helpers.h"

// 2^13
#define NMC  8192 

// MCMC Kernel
__global__  void my_add(int *a, int *b, int *c){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    c[index] = a[index] + b[index];
}

int main(void) 
{
	  int i,j;
    // Allocate and initialize the matrices
    Matrix  M  = AllocateMatrix(WIDTH, WIDTH);
 
    // Initialize Matrix of grid points
    for(unsigned int i = 0; i < M.height; i++)
    {
        for(unsigned int j=0;j < M.width; j++)
        {
            M.elements[i * M.width + j] = (WIDTH/2 - i)*(WIDTH/2 - i) + (WIDTH/2 - j)*(WIDTH/2 - j);
        }
    }
 
	  PrintMatrix(M.elements,M.width,M.height);
  	printf("\n");
 
    // X and Y vectors
    Vector IdxI = AllocateVector(WIDTH);
    Vector IdxJ = AllocateVector(WIDTH);
 
    SerializeVector(IdxI, "random_numbers.dat");
    SerializeMatrix(M, "DistanceMatrix.dat");

    // Vector of measurements
    Vector PiValues = AllocateVector(NMC);
    

    // Free matrices
    FreeMatrix(M);
    FreeVector(IdxI);
    FreeVector(IdxJ);
    FreeVector(PiValues);

    return 0;
}

In [ ]:
!nvcc -I./lib/ -o "/content/drive/My Drive/calculate_pi_GPU/src/calc_pi_MCMC_with_git_files" "/content/drive/My Drive/calculate_pi_GPU/src/calc_pi_MCMC_with_git_files.cu" -lcurand
!"/content/drive/My Drive/calculate_pi_GPU/src/calc_pi_MCMC_with_git_files"

# Now work directly on repo

In [ ]:
!cat src/main.cpp

In [ ]:
/*
 * Author:  Vijay Gopal Chilkuri
 * Email:   vijay.gopal.c@gmail.com
 * Date:    12-08-2020
 */

#include "linear_algebra_helpers.h"

// 2^13
#define NMC  8192 

// MCMC Kernel
__global__  void my_add(int *a, int *b, int *c){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    c[index] = a[index] + b[index];
}

int main(void) 
{
	  int i,j;
    // Allocate and initialize the matrices
    Matrix  M  = AllocateMatrix(WIDTH, WIDTH);
 
    // Initialize Matrix of grid points
    for(unsigned int i = 0; i < M.height; i++)
    {
        for(unsigned int j=0;j < M.width; j++)
        {
            M.elements[i * M.width + j] = (WIDTH/2 - i)*(WIDTH/2 - i) + (WIDTH/2 - j)*(WIDTH/2 - j);
        }
    }
 
	  PrintMatrix(M.elements,M.width,M.height);
  	printf("\n");
 
    // X and Y vectors
    Vector IdxI = AllocateVector(WIDTH);
    Vector IdxJ = AllocateVector(WIDTH);
 
    SerializeVector(IdxI, "random_numbers.dat");
    SerializeMatrix(M, "DistanceMatrix.dat");

    // Vector of measurements
    Vector PiValues = AllocateVector(NMC);
    

    // Free matrices
    FreeMatrix(M);
    FreeVector(IdxI);
    FreeVector(IdxJ);
    FreeVector(PiValues);

    return 0;
}

In [ ]:
!ls

In [ ]:
!mkdir build

In [ ]:
!ls /usr/local/cuda

In [ ]:
%cd build

In [ ]:
!git pull

In [ ]:
!cmake -D CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda ..

In [ ]:
!make

In [ ]:
!ls ../src ../lib ../build ..

In [ ]:
!./calcpigpu

# Work flow 

## 1. Commit on local repo and push

## 2. Pull on colab

In [ ]:
!pwd

In [ ]:
!git pull

## 3. Compile

In [ ]:
!make

## 4. Run

In [ ]:
!./calcpigpu

## 4. Analyse with python

In [ ]:
import matplotlib.pyplot as plt
import numpy

data = numpy.loadtxt("../DistanceMatrix.dat")
size = data.shape
dimM = int(numpy.sqrt(size[0]));
dimN = int(size[0]/dimM)

print(numpy.sqrt(size))
# plt.plot(data)
plt.imshow(data.reshape(dimM,dimN), cmap='hot', interpolation='nearest')
plt.show()